In [ ]:
import polars as pl

battery = "data/huduma_centre_battery_voltage_01May24-16May24.csv"
iqair = "data/IQAir_raw_twrgmdgz53t_16Apr24-17May24_hourly.csv"

df_battery = pl.read_csv(source=battery, try_parse_dates=True)
df_battery = df_battery.with_columns(pl.col("Date").str.strptime(format="%d.%m.%Y %H:%M:%S", dtype=pl.Datetime))
df_battery = df_battery.sort("Date")
df_battery_1h = df_battery.group_by_dynamic("Date", every="1h").agg(pl.col("Voltage").mean().alias("Battery (V)"))
df_battery_1h.describe()

In [ ]:
df_iqair = pl.read_csv(source=iqair, try_parse_dates=True)

# shift timestamps to UTC
df_iqair = df_iqair.with_columns((pl.col("Datetime_start") - pl.duration(hours=2)).alias("Date"),
                                 (pl.col("Datetime_end") - pl.duration(hours=2)),
                                 )
df_iqair = df_iqair.drop(["Timezone", "CO (ppm)", "TVOC (index)", "NOx (index)", "O3 (ug/m3)", "HCHO (ppb)", "TVOC (ppb)"])
df_iqair.describe()
# df_iqair.schema

In [ ]:
df = pl.concat([df_battery_1h, df_iqair], how="diagonal")
display(df)
df.plot.scatter(x="Date", y=["Temperature (Celsius)","Battery (V)", ])